In [1]:
import pandas as pd
import numpy as np
import  re
from tqdm import tqdm 

import pyLDAvis.gensim_models as gensimvis
import numpy as np 
import pandas as pd
import pyLDAvis
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora
import gensim

import pyLDAvis.gensim_models as gensimvis
import scispacy
import spacy
from scipy.spatial.distance import jensenshannon
import en_core_sci_lg
import joblib
from IPython.display import HTML, display
from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Import data

In [19]:
data_scopus = pd.read_csv('./data/scopus/scopus_data.csv', sep=';');
data_scopus.head(1)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Gordon N.K., Gordon R.",57169700700;7402753791;,Embryogenesis explained,2016,Embryogenesis explained,NaN,NaN,NaN,1,759,...,9789814350501; 9789814350488,NaN,NaN,English,Embryog. explain.,Book,Final,NaN,Scopus,2-s2.0-84857183528


In [20]:
data_pubmed = pd.read_csv('./data/pubmed/set_DT.csv', sep=',');
data_pubmed.head(1)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,31738442,How to govern the digital transformation of he...,"Ricciardi W, Pita Barros P, Bourek A, Brouwer ...",Eur J Public Health. 2019 Oct 1;29(Supplement_...,Ricciardi W,Eur J Public Health,2019,2019/11/19,PMC6859522,NaN,10.1093/eurpub/ckz165


## Preparing data for LDA (PubMed + Scopus)

In [26]:
content = ''
with open('./data/pubmed/set_DT.txt') as f:
    for line in f:
        content = content + line

In [27]:
pubmed_dict = {}
PMID = re.findall('\nPMID-(.*)\n', content, re.MULTILINE)
DP = re.findall('\nDP  -(.*)\n', content, re.MULTILINE)
AB = re.findall('\nAB  -(.*)\n', content, re.MULTILINE)
pubmed_dict = {
        'pmid':PMID,
        'date':DP,
        'abstract':AB}
for i in range(len(PMID)-len(AB)):
    AB.append('nan')

In [28]:
data_pubmed = pd.DataFrame(pubmed_dict)
data_pubmed.head()
data_pubmed['pmid']=data_pubmed['pmid'].str.strip()

In [29]:
data_scopus['pmid'] = data_scopus['EID']
data_scopus['abstract'] = data_scopus['Abstract']
data_scopus['date']=data_scopus['Year']

In [59]:
data_scopus.to_csv('./network_data/data_scopus.csv', index=False)

In [30]:
data_pubmed = pd.concat([data_pubmed,data_scopus[['pmid','date','abstract']]])
data_pubmed.tail(5)

,pmid,date,abstract
14338,2-s2.0-85131519147,2022,Introduction: Remotely delivered pain manageme...
14339,2-s2.0-85131525515,2022,The pilot survey presented herein explores the...
14340,2-s2.0-85131545214,2022,[No abstract available]
14341,2-s2.0-85131563626,2022,The COVID 19 (Coronavirus) pandemic has led to...
14342,2-s2.0-85131583685,2022,Cancer remains a common health issue having si...


In [31]:
data_pubmed.shape

(20892, 3)

In [32]:
parser = en_core_sci_lg.load(disable=["tagger", "ner"])
parser.max_length = 6000000

In [33]:
customize_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 'CZI',
    '-PRON-','nan','hz','new','use','image','images','methods','method','study','studies','area'
]
for w in customize_stop_words:
    parser.vocab[w].is_stop = True

In [34]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [ word.lemma_.lower().strip() for word in tokens if not(word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1) ]
    tokens = " ".join([i for i in tokens])
    return tokens

In [35]:
tqdm.pandas()
data_pubmed["processed_abstract"] = data_pubmed['abstract'].progress_apply(spacy_tokenizer)

  0%|                                                 | 0/20892 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|█████████████████████████████████████| 20892/20892 [09:56<00:00, 35.00it/s]


In [ ]:
docs_clean = []
for i in list(data_pubmed.processed_abstract):
    docs_clean.append(i.split())
id2word = corpora.Dictionary(docs_clean)

In [36]:
cvec = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
vectorized_data = cvec.fit_transform(tqdm(data_pubmed['processed_abstract']))

<>:1: DeprecationWarning: invalid escape sequence \-
<>:1: DeprecationWarning: invalid escape sequence \-
/var/folders/0d/7rwsnbd936d6v32kctnxnw4w0000gn/T/ipykernel_25968/2908419801.py:1: DeprecationWarning: invalid escape sequence \-
  cvec = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
100%|██████████████████████████████████| 20892/20892 [00:01<00:00, 15233.83it/s]


In [37]:
vectorized_data.shape

(20892, 20393)

# LDA modeling

In [54]:
lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(vectorized_data)

LatentDirichletAllocation(n_components=20, random_state=0)

In [55]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [56]:
print_top_words(lda, cvec, n_top_words=40)


Topic #0: proposed algorithm detection based results paper transform features color wavelet performance classification digital compression feature different technique accuracy retinal information analysis breast approach scheme diagnosis techniques segmentation medical quality compared data database noise obtained algorithms fundus region extracted applied regions

Topic #1: signal frequency signals analysis data transform digital eeg fourier time ecg power spectral processing results based filter fast noise flow control detection obtained heart rate recorded paper spectrum information parameters proposed different wavelet frequencies algorithm spectra high function monitoring domain

Topic #2: model data learning segmentation models features network proposed based classification results deep performance accuracy machine approach neural analysis training different dataset datasets feature techniques information algorithms set medical networks prediction paper detection approaches auto

In [57]:
doc_topic_dist = pd.DataFrame(lda.transform(vectorized_data))
doc_topic_dist.columns = ["topic_"+str(column) for column in doc_topic_dist.columns.values]
df = data_pubmed.join(doc_topic_dist)

In [58]:
df.to_csv('./network_data/df.csv', index=False)